In [3]:
import sys
sys.path.append("..")


In [7]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms  # type: ignore
from torchvision.datasets import CIFAR10  # type: ignore

from src.models.bi_sngan import BiSNGAN, Discriminator32, Encoder32, Generator32


In [8]:
BATCH_SIZE = 64
EPOCHS = 200
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [15]:
transform = transforms.Compose(
    [
        transforms.Lambda(lambd=lambda i: i.to(DEVICE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=(0.5, 0.5, 0.5)),
    ]
)


In [11]:
dataset = CIFAR10(root="dataset", download=True, transform=transform)
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)


100%|██████████| 170498071/170498071 [00:39<00:00, 4268633.95it/s] 


Extracting dataset/cifar-10-python.tar.gz to dataset


In [ ]:
generator = Generator32(output_channels=3)
encoder = Encoder32(input_channels=3)
discriminator = Discriminator32(input_channels=3)

model = BiSNGAN(encoder=encoder, generator=generator, discriminator=discriminator)
model = model.to(DEVICE)

for epoch in range(EPOCHS):
    ge_loss, disc_loss = model.train_single_epoch(train_loader=dataloader)
